In [1]:
from src import data_loader as dl
import pandas as pd

file_path = 'data/U13271915_20250101_20251029.csv'
df_raw = pd.read_csv(
            file_path,
            header=None,
            dtype=object, 
            on_bad_lines='skip',
            low_memory=False
        )
print("df_raw loaded successfully!")
print(df_raw.head)

df_raw loaded successfully!
<bound method NDFrame.head of                     0       1   \
0            Statement  Header   
1            Statement    Data   
2            Statement    Data   
3            Statement    Data   
4            Statement    Data   
..                 ...     ...   
713  Notes/Legal Notes    Data   
714  Notes/Legal Notes    Data   
715  Notes/Legal Notes    Data   
716  Notes/Legal Notes    Data   
717  Notes/Legal Notes    Data   

                                                    2   \
0                                           Field Name   
1                                           BrokerName   
2                                        BrokerAddress   
3                                                Title   
4                                               Period   
..                                                 ...   
713  Over-the-Counter Transactions in Contracts for...   
714  Over-the-Counter Transactions in Contracts for...   
715  Over-t

In [2]:
df_mtm = dl.get_section(df_raw, 'Mark-to-Market Performance Summary')
print("--- MTM Section ---")
print(df_mtm)

df_open_pos = dl.get_section(df_raw, 'Open Positions')
print("\n--- Open Positions Section ---")
print(df_open_pos)

--- MTM Section ---
37                     Asset Category    Symbol Prior Quantity  \
0                              Stocks      ANFO              0   
1                              Stocks      ASML              2   
2                              Stocks    CHDVDz             45   
3                              Stocks       DGE             56   
4                              Stocks      EMBC              0   
5                              Stocks      FLIN            192   
6                              Stocks      FONC              0   
7                              Stocks        HO              0   
8                              Stocks      IBKR         3.0368   
9                              Stocks      IGLB              0   
10                             Stocks     IHYUz              0   
11                             Stocks     IUSCz             86   
12                             Stocks       IVV             12   
13                             Stocks      KGXd         

In [3]:
import pandas as pd
# Assuming your loader file is in a folder named 'src'
# and this script is in the main project folder.
from src import data_loader as dl 

# --- 1. Define File Paths ---
# Make sure this path is correct for your project structure
filepath = r'data/U13271915_20250101_20251029.csv'
output_excel_path = r'data/report_output.xlsx'

# --- 2. Run the Loader ---
# Set pandas display options for better printing in the console
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 150)

print(f"Loading report from: {filepath}...")
results = dl.load_ibkr_report(filepath)


# --- 3. Save to Excel and Print ---
if results:
    print("\nReport loaded successfully.")
    
    try:
        # Use pd.ExcelWriter to save both DataFrames to one file
        # on separate sheets.
        with pd.ExcelWriter(output_excel_path) as writer:
            results['initial_state'].to_excel(writer, sheet_name='Initial_State', index=False)
            results['events'].to_excel(writer, sheet_name='Event_Log', index=False)
            
        print(f"Successfully saved data to: {output_excel_path}")
        
    except ImportError:
        print("\n--- ERROR ---")
        print("To save to Excel, you need the 'openpyxl' library.")
        print("Please install it by running this in your terminal:")
        print("pip install openpyxl")
    except Exception as e:
        print(f"\nAn error occurred while saving to Excel: {e}")

    # --- 4. Print results to console as before ---
    print("\n" + "="*80 + "\n")
    print("--- 1. Initial State DataFrame ---")
    print(results['initial_state'])
    
    print("\n" + "="*80 + "\n")
    print("--- 2. Master Event Log (First 20 Events) ---")
    print(results['events'].head(20))

else:
    print("Data loading failed. No results to save.")

Loading report from: data/U13271915_20250101_20251029.csv...

Report loaded successfully.
Successfully saved data to: data/report_output.xlsx


--- 1. Initial State DataFrame ---
    symbol asset_category currency      quantity  value_native
0     ANFO          Stock      CHF      0.000000      0.000000
1     ASML          Stock      USD      2.000000   1386.160000
2   CHDVDz          Stock      CHF     45.000000   7106.400000
3      DGE          Stock      GBP     56.000000   1421.000000
4     EMBC          Stock      CHF      0.000000      0.000000
5     FLIN          Stock      USD    192.000000   7276.800000
6     FONC          Stock      CHF      0.000000      0.000000
7       HO          Stock      EUR      0.000000      0.000000
8     IBKR          Stock      USD      3.036800    536.511456
9    IHYUz          Stock      CHF      0.000000      0.000000
10   IUSCz          Stock      CHF     86.000000   7833.740000
11     IVV          Stock      USD     12.000000   7064.160000
12

In [4]:
dl.get_section(df_raw, 'Corporate Actions')

280,Asset Category,Currency,Report Date,Date/Time,Description,Quantity,Proceeds,Value,Realized P/L
0,Stocks,EUR,29/05/2025,"2025-05-28, 20:25:00",VNA(DE000A1ML7J1) Dividend Rights Issue 1 for...,136,0,0,0
1,Stocks,EUR,30/06/2025,"2025-06-25, 20:20:00",VNA.DRTS(DE000A40ZU23) Expire Dividend Right (...,-136,0,0,0
2,Stocks,EUR,30/06/2025,"2025-06-25, 20:20:00",VNA.DRTS(DE000A40ZU23) Expire Dividend Right (...,5,0,0,0
3,Total,NaN,NaN,NaN,NaN,NaN,0,0,0
4,Total in CHF,NaN,NaN,NaN,NaN,NaN,0,0,0
5,Stocks,USD,18/06/2025,"2025-06-17, 20:25:00","IBKR(US45841N1072) Split 4 for 1 (IBKR, INTERA...",9.1104,0,0,0
6,Total,NaN,NaN,NaN,NaN,NaN,0,0,0
7,Total in CHF,NaN,NaN,NaN,NaN,NaN,0,0,0
